In [10]:
from Explanations_Models.Custom_DT.LIME import LIME
import yaml
import argparse
import os
import sys
import numpy as np
import torch
import gym
import matplotlib
import matplotlib.pyplot as plt
import unittest
from DeepLearning_Models.utils.general import join, plot_combined
from DeepLearning_Models.ActorCritic.policy_gradient import PolicyGradient
from EnvRunner import GymRunner
import json
import warnings
from tqdm import tqdm

# Suppress all deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

yaml.add_constructor("!join", join)
parser = argparse.ArgumentParser()

parser.add_argument("--config_filename", required=False, type=str)

_StoreAction(option_strings=['--config_filename'], dest='config_filename', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help=None, metavar=None)

In [11]:

config_file = open("config_envs/{}.yml".format("cartpole"))
config = yaml.load(config_file, Loader=yaml.FullLoader)
config.update(yaml.load(open("config_explanations/{}.yml".format("cartpole"), encoding="utf8"), Loader= yaml.FullLoader))
Runner = GymRunner(config)
Runner.load_weights(PATH = None)


device:  cpu


c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\EnvRunner.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.baseline_network.load_state_

In [12]:
DT = LIME(config, Runner)

In [13]:
X,Y = DT.train(returner=True)


60


In [14]:
DT.percent_Correct(print_val=True)


Percent Correct: 0.81%


0.805

In [ ]:
DT.uniform_Correct(print_val=True)

In [15]:
DT.surr_model.model.get_dict_representation()

{'Feature': 'angle_vel',
 'Bucket': 0.11588563770055771,
 'Left_Child': {'Feature': 'angle_vel',
  'Bucket': -0.0694764256477356,
  'Left_Child': {'Feature': 'x',
   'Bucket': -1.1615281105041504,
   'Left_Child': {'Feature': 'x',
    'Bucket': -1.241638422012329,
    'Left_Child': {'Value': 0},
    'Right_Child': {'Value': 1}},
   'Right_Child': {'Value': 0}},
  'Right_Child': {'Feature': 'x',
   'Bucket': -0.28901487588882446,
   'Left_Child': {'Feature': 'angle',
    'Bucket': -0.016896795481443405,
    'Left_Child': {'Value': 0},
    'Right_Child': {'Value': 1}},
   'Right_Child': {'Feature': 'x',
    'Bucket': 0.02948625013232231,
    'Left_Child': {'Value': 0},
    'Right_Child': {'Value': 1}}}},
 'Right_Child': {'Value': 1}}

In [9]:
np.mean(Y, axis=0)

array([ 0.74338526,  1.4490379 ,  1.0475755 , -0.23259966], dtype=float32)

In [ ]:
from scipy.spatial.distance import pdist
import pandas as pd
Y = pd.DataFrame(Y, columns=config["picture"]["class_names"])
np.mean(pdist(Y.values, metric="euclidean"))

In [9]:
import pandas as pd
Y = np.array([[1., 4.2, 3.3]])
Y = pd.DataFrame(Y, columns = ["hit1", "hit2", "hit3"])

In [ ]:
from scipy.spatial.distance import pdist

np.mean(pdist(Y.values, metric="euclidean"))